<a href="https://colab.research.google.com/github/adamb2003/FPL/blob/main/fplsolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FPL Python Solver - Google Colab version

**Notes**

- You need a prediction data (CSV) file to use this tool, either from FPLReview, Mikkel Tokvam, or The FPL Kiwi.
- *First time here?* Check my step-by-step tutorial on YouTube: https://youtu.be/bOiCmines3M
- You can click "Runtime" and select "Disconnect and Delete Runtime" to clear all files if you are not running this solver for the first time and having issues.
- You need to run both blue colored 🔽 and red colored 🔻 cells if you are running this solver for the first time. If you already have the solver, you can run red colored 🔻 cells only.
- Questions/feedbacks/comments? Send me a DM on Twitter [@sertalpbilal](https://twitter.com/messages/compose?recipient_id=14057207)

In [ ]:
#@title 🔽 Run this cell to download the optimization solver (HiGHS)
%%bash
if [ -e solver ]
then
    echo "Solver is already installed, continue..."
else
    echo "Downloading solver"
    apt-get install -qq wget
    wget -nv https://github.com/JuliaBinaryWrappers/HiGHSstatic_jll.jl/releases/download/HiGHSstatic-v1.4.2%2B0/HiGHSstatic.v1.4.2.x86_64-linux-gnu-cxx11.tar.gz
    mkdir solver
    tar xzf HiGHSstatic.v1.4.2.x86_64-linux-gnu-cxx11.tar.gz -C solver
    echo "Done..."
fi

In [ ]:
#@title 🔻 Run this cell to download (or update) FPL Optimization codes
%%bash
if [ -e FPL-Optimization-Tools ]
then
    echo "Removing old codes"
    rm -rf FPL-Optimization-Tools
else
    echo "Downloading codes for the first time"
fi
echo "Cloning..."
git clone -q https://github.com/sertalpbilal/FPL-Optimization-Tools.git
pip install -q -r FPL-Optimization-Tools/requirements.txt
echo "Done!"

In [ ]:
#@title 🔻 Enter your team ID below and run this cell

team_id =   32409 #@param {type:"integer"}

print(f"Now visit https://fantasy.premierleague.com/api/my-team/{team_id}/ and copy the content")

In [ ]:
#@title 🔻 Run this cell first, and once you see a textbox, paste the content of the URL to the textbox below.
!pip install -q ipywidgets
import ipywidgets as widgets
team_data = widgets.Textarea(
    value='',
    placeholder='Put my-team URL content here',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width="500px", height="100px")
)
team_data

In [ ]:
#@title 🔻 Run this cell to parse your team information you pasted into the program.

import json
team_values = json.loads(team_data.value)
with open('team.json', 'w') as f:
  json.dump(team_values, f, indent=2)
# team_values
print("Done!")

In [ ]:
import pathlib
data_path = pathlib.Path('./data')
data_path.mkdir(exist_ok=True)
parent_data_path = pathlib.Path('../data')
parent_data_path.mkdir(exist_ok=True)
#@title 🔻 Run this cell to create the data folder, then:
#@markdown - Click Files (📁) on the left
#@markdown - Right click on an empty space and select "Refresh".
#@markdown - Navigate to `data` directory.
#@markdown - Right click on directory and upload FPLReview projections (should be named **fplreview.csv**)
#@markdown - The name should be **kiwi.csv** if you are using TheFPLKiwi's projections, or **mikkel.csv** for Mikkel Tokvam's


In [11]:
import sys
sys.path.append("FPL-Optimization-Tools/src")

import subprocess
if 'o_PIPE' not in locals():
  o_PIPE = subprocess.PIPE
  o_POPEN = subprocess.Popen

def realtime_popen(cmd, **kwargs):
  with o_POPEN(cmd.split(), stdout=o_PIPE, bufsize=1, universal_newlines=True) as p:
    for line in p.stdout:
      print(line, end='')
    return p

subprocess.Popen = realtime_popen

from multi_period_dev import prep_data, solve_multi_period_fpl
#@title 🔻 Run this cell to import solver and redirect log

In [12]:
options = {
    "horizon": 5,
    "ft_value": 1.5,
    "itb_value": 0.05,
    "no_transfer_last_gws": 2,
    "xmin_lb": 20,
    "ev_per_price_cutoff": 20,
    "no_future_transfer": False,
    "num_transfers": None,
    "secs": 300,
    "banned": [],
    "locked": [],
    "use_wc": None,
    "use_fh": None,
    "use_bb": None,
    "chip_limits": {"bb": 0, "wc": 0, "fh": 0, "tc": 0},
    "no_chip_gws": [],
    "allowed_chip_gws": {"bb": [], "wc": [], "fh": [], "tc": []},
    "no_gk_rotation_after": None,
    "iteration": 1,
    # Replace with 'kiwi' if you are using Kiwi's data, 
    # or 'mikkel' if you are using Mikkel Tokvam's data
    'datasource': 'kiwi',
    # do not edit parameters below this line
    'data_path': './data/fplreview.csv',
    'kiwi_data_path': './data/kiwi.csv',
    'mikkel_data_path': './data/mikkel.csv',
    'solver': 'highs',
    'solver_path': './solver/bin/highs',
    'presolve': 'on',
    'use_cmd': False
}
data = prep_data(team_values, options)
result = solve_multi_period_fpl(data, options)
print("Done!")
#@title 🔻 Edit settings inside this cell and run it to start the solver

579 total players (before)
352 total players (after)
Added player 81 to list, buy price 4.9 sell price 4.8
Added player 357 to list, buy price 6.2 sell price 5.7
Added player 283 to list, buy price 12.9 sell price 12.8
Added player 107 to list, buy price 5.2 sell price 5.1
Added player 133 to list, buy price 4.7 sell price 4.6
Added player 586 to list, buy price 4.8 sell price 4.7
Added player 335 to list, buy price 7.1 sell price 6.9
NOTE: Initialized model mp_h5_od_d0.84.
Exported problem with name: mp_h5_od_d0.84_pBFx9_0
Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Number of PL entries in BOUNDS section is 5
Number of BV entries in BOUNDS section is 19767
MIP  mp_h5_od_d0.84_pBFx9_0 has 34245 rows; 19784 cols; 131547 nonzeros; 19778 integer variables
Presolving model
18999 rows, 16940 cols, 73589 nonzeros
17714 rows, 15423 cols, 65836 nonzeros

Solving MIP model with:
   17714 rows
   15423 cols (15416 binary, 4 int

In [13]:
#@title 🔻 Run this cell to display solutions



import pandas as pd
import datetime
import os
from IPython.display import display, HTML, Markdown, Javascript

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

for r in result:
  iter = r['iter']
  time_now = datetime.datetime.now()
  stamp = time_now.strftime("%Y-%m-%d_%H-%M-%S")
  if not (os.path.exists("../data/results/")):
    pathlib.Path("./data/results/").mkdir(parents=True, exist_ok=True)
  r['picks'].to_csv(f"./data/results/regular_{stamp}_{iter}.csv")

result_table = pd.DataFrame(result)
rt = result_table[['iter', 'sell', 'buy', 'score']].copy()
rt['iter'] += 1

display(Markdown(f'## Solution Summary'))
display(HTML(rt.to_html()))

display(Markdown(f'## Solution Details'))

for j, r in enumerate(result):
  display(Markdown(f'### Solution {j+1}'))
  print(r['summary'])


<IPython.core.display.Javascript object>

## Solution Summary

,iter,sell,buy,score
0,1,-,-,277.742369


## Solution Details

### Solution 1

** GW 33:
ITB=0.7, FT=1, PT=0, NT=0
---
Lineup: 
	Arrizabalaga (3.95)
	Schär (3.79), Estupiñán (3.84), Trippier (4.03)
	Rashford (3.94), Fernandes (4.03), Mitoma (4.17), March (4.18), Salah (5.1, V)
	Darwin (3.49), Haaland (5.26, C)
Bench: 
	Raya, Chilwell, Shaw, Greenwood
Lineup xPts: 51.04
---

** GW 34:
ITB=0.4, FT=2, PT=0, NT=2
Buy 307 - Ederson
Buy 309 - Laporte
Sell 81 - Raya
Sell 139 - Chilwell
---
Lineup: 
	Ederson (8.35)
	Laporte (6.99), Shaw (7.18), Estupiñán (7.99)
	Mitoma (8.2), March (8.39), Rashford (8.91), Fernandes (9.15), Salah (11.8, V)
	Darwin (8.37), Haaland (12.21, C)
Bench: 
	Arrizabalaga, Trippier, Schär, Greenwood
Lineup xPts: 109.75
---

** GW 35:
ITB=0.5, FT=1, PT=0, NT=1
Buy 116 - Mac Allister
Sell 124 - Mitoma
---
Lineup: 
	Ederson (4.34)
	Trippier (3.64), Laporte (3.72), Estupiñán (4.33)
	Rashford (4.33), Fernandes (4.43), March (4.47), Mac Allister (4.92), Salah (5.66, V)
	Darwin (3.83), Haaland (6.9, C)
Bench: 
	Arrizabalaga, Schär, Shaw, Greenwood
Lineup